In [ ]:
!pip install langchain
!pip install openai
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

# 새 섹션

In [ ]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template= "You are a helpful assistant that make an argument for numexpr.evaluate according to user query such as\n```text\n13**0.3432\n``` when user query is 'What is 13 raised to the .3432 power?' Please response as code block using text back quote like ```text\n13**0.3432\n``` without other description."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_model = ChatOpenAI(temperature=0)

chain = chat_prompt_template | chat_model

chain.invoke({"text" : "What is 15 raised to the .3432 power?"})

AIMessage(content='```text\n15**0.3432\n```', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 96, 'total_tokens': 107}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [ ]:
from langchain.schema import BaseOutputParser

class CodeOutputParser(BaseOutputParser):
    def parse(self, text: str):
        value = text.strip().split("```")
        print(value)
        return value[1].split("\n")[1]

chain = chat_prompt_template | chat_model | CodeOutputParser()

output = chain.invoke({"text" : "What is 15 raised to the .3432 power?"})

print(output)

['', 'text\n15**0.3432\n', '']
15**0.3432


In [ ]:
import numexpr

In [ ]:
numexpr.evaluate('15**0.3432')

array(2.53299609)

In [ ]:
chain = chat_prompt_template | chat_model | CodeOutputParser() | numexpr.evaluate

output = chain.invoke({"text" : "What is 15 raised to the .3432 power?"})

print(output)

['', 'text\n15**0.3432\n', '']
2.5329960941138205


In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

chain = ({"text" : RunnablePassthrough() }
        | chat_prompt_template
        | chat_model
        | CodeOutputParser()
        | numexpr.evaluate)

output = chain.invoke("What is 15 raised to the .3432 power?")

print(output)

['', 'text\n15**0.3432\n', '']
2.5329960941138205


In [ ]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

explain_template= """You are a helpful math assistant that detailed explain the following problem and solution in Korean.

problem : {problem}
solution : {solution}
output : {output}"""

explain_human_message_prompt = HumanMessagePromptTemplate.from_template(explain_template)

explain_chat_prompt_template = ChatPromptTemplate.from_messages([explain_human_message_prompt])

explain_chain = ({"text" : RunnablePassthrough()}
                 | RunnablePassthrough.assign(solution = chat_prompt_template | chat_model | CodeOutputParser())
                 | RunnablePassthrough.assign(output = itemgetter("solution") | RunnableLambda(numexpr.evaluate),
                                problem = RunnablePassthrough())
                 | explain_chat_prompt_template
                 | chat_model
                 | StrOutputParser())

explain_chain.invoke("What is 15 raised to the .3432 power?")

['', 'text\n15**0.3432\n', '']


'주어진 문제는 "15을 0.3432 제곱한 값은 얼마입니까?" 입니다.\n\n해결 방법은 다음과 같습니다:\n15의 0.3432 제곱을 계산하면 됩니다. 이를 파이썬 코드로 나타내면 15**0.3432가 됩니다.\n\n따라서, 15을 0.3432 제곱한 값은 약 2.533입니다.'

In [ ]:
'''
"You are a helpful assistant that make an argument for numexpr.
evaluate according to user query such as\n```text\n13**0.3432\n```
 when user query is 'What is 13 raised to the .3432 power?'
 Please response as code block using text back quote like ```text\n13**0.3432\n```
 without other description."
'''

my_math_chain = LLMChain(llm=chat_model,
                         prompt=chat_prompt_template,
                         output_parser=CodeOutputParser(),
                         verbose=True)

output = my_math_chain.run(text="What is 15 raised to the .3432 power?")

print(numexpr.evaluate(output))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant that make an argument for numexpr.evaluate according to user query such as
```text
13**0.3432
``` when user query is 'What is 13 raised to the .3432 power?' Please response as code block using text back quote like ```text
13**0.3432
``` without other description.
Human: What is 15 raised to the .3432 power?
['', 'text\n15**0.3432\n', '']

> Finished chain.
2.5329960941138205
